In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install catboost
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

import tqdm
import gc
import math

import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime, timedelta
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

# deeplearning
import tensorflow as tf
from tensorflow.keras import layers  
# from sklearn import tree
# from sklearn.tree import export_graphviz

# machine learning
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

# regression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

# svm
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 
from catboost import Pool,CatBoostClassifier

import os
import pandas as pd
import numpy as np

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Input, Dense, Activation, Reshape, Concatenate
from keras.layers.embeddings import Embedding
import matplotlib.pyplot as plt
from keras.models import Model as KerasModel
import bisect

     |████████████████████████████████| 76.1 MB 1.6 MB/s 


In [3]:
path="/gdrive/My Drive/Colab Notebooks/dacon/job_care_data"

ori_train = pd.read_csv(path+'/train.csv')
ori_test = pd.read_csv(path+'/test.csv')
sample_submission = pd.read_csv(path+'/sample_submission.csv')

train_size = len(ori_train)
test_size = len(ori_test)

display(ori_train,ori_test)

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target
0,0,True,True,True,False,False,False,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1
1,1,False,False,False,True,True,False,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0
2,2,False,False,False,True,False,False,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0
3,3,False,False,False,True,False,False,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0
4,4,True,True,True,False,False,False,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,501946,False,False,False,True,False,False,1,1,2,2,1192,935,1228,3,1,1,59,4,95,3,3,5,1,1,2,354,147,1,5,65,503156,285850,2020-03-13 12:55:52,1
501947,501947,True,True,False,True,False,False,1,6,2,1,118,113,110,4,1,1,105,142,95,3,3,10,2,1,2,163,120,1,4,142,676255,456996,2020-01-20 11:51:51,1
501948,501948,True,True,True,True,False,False,1,7,4,1,147,46,145,4,1,1,59,127,139,3,1,5,1,1,2,438,147,2,7,65,484528,293258,2020-08-05 17:27:24,1
501949,501949,True,False,False,True,False,False,1,1,2,1,46,147,145,4,1,1,251,49,258,3,2,5,1,1,2,660,147,3,4,259,456330,273797,2020-06-15 09:23:21,1


,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt
0,0,True,False,False,True,True,True,1,1,2,1,857,851,1227,4,1,1,263,56,49,1,3,10,2,1,2,1147,839,1,5,263,393790,236865,2020-12-01 02:24:18
1,1,False,False,False,True,False,False,2,0,2,2,683,1086,662,2,1,1,258,263,49,1,3,5,1,1,2,1611,278,1,4,263,394058,236572,2020-12-17 05:42:53
2,2,True,False,False,True,True,True,2,3,2,1,514,790,1233,0,1,1,177,170,171,3,3,3,1,1,2,1817,490,3,4,177,1002061,704612,2020-12-10 23:33:41
3,3,True,False,False,True,True,True,1,2,2,5,114,181,175,4,1,1,177,170,171,3,3,5,1,1,2,101,150,5,3,177,1000813,704652,2020-12-03 19:44:55
4,4,True,False,False,True,False,False,1,6,4,5,1082,1078,1056,5,1,1,178,177,4,3,1,5,1,1,1,985,1097,1,4,177,111146,704413,2020-12-11 21:24:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,46399,True,True,True,False,False,False,2,0,4,1,147,46,145,4,1,1,2,4,95,3,3,5,1,1,2,759,147,1,5,91,425040,726084,2020-12-17 01:06:26
46400,46400,True,False,False,True,False,False,2,0,4,1,176,120,159,4,1,1,86,31,278,3,3,10,2,1,2,759,147,1,4,91,290061,156948,2020-12-29 21:57:25
46401,46401,True,True,False,True,True,True,2,0,3,1,145,46,147,5,1,1,288,279,278,3,3,10,2,1,2,759,147,1,4,288,307951,175069,2020-12-10 19:09:09
46402,46402,True,True,False,True,True,True,1,3,3,2,145,46,147,6,1,1,288,279,278,3,1,5,1,3,2,759,147,1,5,288,308354,174849,2020-12-07 14:04:34


# [전처리함수]

In [4]:
def check_diff(train,test):
    '''
    column별 test에만 있는 values들을 확인한다
    '''
    cols = test.columns
    
    for col in cols:
        print(col)
        print(set(test[col]) - set(train[col]))
        print(set(test[col]) == set(train[col]))

def do_onehot_v2(train,test,cols):
    '''
    df : data frame
    cols : names of columns (type:list)
    one hot encode cols in train and test set
    col name rule : {original col name}:dummy:{dummy values}
    '''
    for col in cols:
        # dummy
        train_dummy_df = pd.get_dummies(train[col])
        test_dummy_df = pd.get_dummies(test[col])

        # fill 0 which exist only in test set
        test_dummy_df = test_dummy_df.reindex(columns = train_dummy_df.columns, fill_value=0)

        # delete origin col
        train = train.drop([col],axis=1)
        test = test.drop([col],axis=1)

        # change col name type {origin col name}:dummy:{value}
        train_dummy_df.columns = [col+":dummy:"+str(val) for val in train_dummy_df.columns]
        test_dummy_df.columns = train_dummy_df.columns

        # concat 
        train = pd.concat([train,train_dummy_df],axis=1)
        test = pd.concat([test,test_dummy_df],axis=1)

    return train,test

def do_onehot(train,test,cols):
    '''
    df : data frame
    cols : names of columns (type:list)
    one hot encode cols in train and test set
    col name rule : {original col name}:dummy:{dummy values}
    '''
    train_size = len(train)
    test_size = len(test)

    for col in cols:
        # [train]
        train_dummy_df = pd.get_dummies(train[col]) # dummy
        
        val_to_col_name_dict = {}
        for val in train_dummy_df.columns:
            val_to_col_name_dict[val] = col+":dummy:"+str(val)

        train_dummy_df.columns = [val_to_col_name_dict[val] for val in train_dummy_df.columns] # column name
        train = train.drop([col],axis=1) # delete original col
        train = pd.concat([train,train_dummy_df],axis=1) # concat

        # [test]
        test_dummy_df = pd.DataFrame({dummy_col:[0 for i in range(test_size)] for dummy_col in train_dummy_df.columns})
        test = pd.concat([test,test_dummy_df],axis=1) # concat

        for val,col_name in val_to_col_name_dict.items():
            test.loc[test[col]==val, col_name] = 1 # test의 col값이 valdlaus col_name은 1

        test = test.drop([col],axis=1) # delete original col

    return (train,test)

def do_binary_encoding(df,cols):
    '''
    df : 데이터프레임
    cols : columns 이름을 값으로 가지는 리스트
    df의 columns를 모두 binary encoding한다.
    '''
    for col in cols:
        # binary df 만들기
        vals = df[col].unique()
        df_size = len(df)
        full_binary_size = len(bin(len(vals)-1)[2:])

        to_binary_dict = {}
        for i,v in enumerate(vals):
            to_binary_dict[v]=bin(i)[2:].zfill(full_binary_size)

        d = []
        for i in range(df_size):
            d.append(list(to_binary_dict[train[col][i]]))

        binary_df = pd.DataFrame(d)
        binary_df.columns = [col+"_binary_"+str(col_name) for col_name in binary_df.columns]
        
        # 기존컬럼 삭제
        df = df.drop([col],axis=1)

        # dummy col붙이기
        df = pd.concat([df,binary_df],axis=1)

    return df

def do_scaling(train,test,cols,type):
    from sklearn import preprocessing
    '''
    scaling by 'type'
    '''

    train_copy = train.copy()
    test_copy = test.copy()

    if type=="minmax":
        scaler = preprocessing.MinMaxScaler()
    elif type=="standard":
        scaler = preprocessing.StandardScaler()
    elif type=="robust":
        scaler = preprocessing.RobustScaler()
    elif type=="normalize":
        scaler = preprocessing.Normalizer()

    scaler_fit = scaler.fit(train_copy[cols])
    
    # [train]
    result=scaler_fit.transform(train_copy[cols])
    for j,col_name in enumerate(cols):
        train_copy[col_name] = result[:,j]

    # [test]
    result_te=scaler_fit.transform(test_copy[cols])
    for j,col_name in enumerate(cols):
        test_copy[col_name] = result_te[:,j]

    return train_copy,test_copy

def do_label_encoding(train,test,cols):
    '''
    label encoding cols in train and transform by train
    '''
    from sklearn.preprocessing import LabelEncoder
    train_copy = train.copy()
    test_copy = test.copy()
    
    for col in cols:
        le=LabelEncoder()
        le.fit(train_copy[col])
        
        train_copy[col] = le.transform(train_copy[col])
        test_copy[col] = le.transform(test_copy[col])


    return (train_copy,test_copy)

def time_labeling(df,col,target_time):
    from datetime import datetime
    '''
    col : str type time column( ex : "2020-07-08 20:00:10", type:str)
    targer_time : target time to convert (ex:["year","month","day","weekday","hour"])
    to onehot train, test time col
    '''

    copied_df = df.copy()
    df_size = len(copied_df)

    time_df={target:[] for target in target_time}
    for i in range(df_size):
        dt = datetime.strptime(copied_df[col][i],"%Y-%m-%d %H:%M:%S")
        if "year" in time_df:
            time_df["year"].append(dt.year)
        if "month" in time_df:
            time_df["month"].append(dt.month)
        if "day" in time_df:
            time_df["day"].append(dt.day)
        if "weekday" in time_df:
            time_df["weekday"].append(dt.weekday())
        if "hour" in time_df:
            time_df["hour"].append(dt.hour)
    time_df=pd.DataFrame(time_df)

    copied_df = copied_df.drop(columns=[col],axis=1)
    copied_df = pd.concat([copied_df,time_df],axis=1)
    
    return copied_df

def convert_to_freq(train_x,train_y,test_x,cols):
    '''
    convert col to label freq
    col 컬럼의 도메인별 label(train_y)의 비율을 계산하고 해당값을 이로 원핫코딩으로 대채한다.
    - label col은 한개여야 한다. 
    - classification 이여야 한다. 
    '''

    train = pd.concat([train_x,train_y],axis=1) 
    train_size = len(train)
    test_size = len(test_x)

    label_col = train_y.columns[0] # label의 컬럼이름 
    label_domains = train_y[label_col].unique() # label domain들 

    for col in cols:
        domain_total = dict(train_x[col].value_counts()) # 도메인별 총개수
        domain_freq_total = {}
        domain_size = len(domain_total)

        for k in domain_total:
            domain_freq = {}
            d_t = domain_total[k] # 해당 도메인의 총개수 
            label_count = dict(train.loc[(train[col]==k)]["target"].value_counts()) # 도메인별 label별 개수 

            for l_d in label_domains:
                domain_freq[l_d] = label_count[l_d] / d_t

            domain_freq_total[k] = domain_freq
        
        # label_domains별 평균(train에는 없고 test에만 있는 값을 대체하기 위해)
        label_domain_average = {label_domain:0 for label_domain in label_domains}
        for v in domain_freq_total.values():
            for label_domain,v in v.items():
                label_domain_average[label_domain] += v

        for label_domain in label_domains:
            label_domain_average[label_domain] = label_domain_average[label_domain]/domain_size

        # [train]
        freq_onehot_dict = {domain:[0 for i in range(train_size)] for domain in label_domains} 
        for i in range(train_size):
            d = train_x[col][i]
            for l in label_domains:
                freq_onehot_dict[l][i] = domain_freq_total[d][l]
        freq_onehot_df = pd.DataFrame(freq_onehot_dict)
        freq_onehot_df.columns = ["freq_onehot_"+col+":"+str(d) for d in freq_onehot_dict]

        # [test]
        freq_onehot_dict_test = {domain:[0 for i in range(test_size)] for domain in label_domains} 
        for i in range(test_size):
            d = test_x[col][i]
            for l in label_domains:
                if d not in domain_freq_total: # train에는 없는 domain값이면 평균으로 대체한다.
                    freq_onehot_dict_test[l][i] = label_domain_average[l]
                else:
                    freq_onehot_dict_test[l][i] = domain_freq_total[d][l]
        freq_onehot_df_test = pd.DataFrame(freq_onehot_dict_test)
        freq_onehot_df_test.columns = ["freq_onehot_"+col+":"+str(d) for d in freq_onehot_dict]

        train_x = pd.concat([train_x,freq_onehot_df],axis=1)
        test_x = pd.concat([test_x,freq_onehot_df_test],axis=1)

        train_x = train_x.drop([col],axis=1)
        test_x = test_x.drop([col],axis=1)

    return train_x, test_x    

def del_one_domain(train_x,test_x):
    '''
    train_x에 도메인이 하나인 컬럼을 삭제한다
    '''
    del_list = []
    cols = train_x.columns
    for col in cols:
        if len(list(train_x[col].unique()))==1:
            del_list.append(col)

    train_x_copy = train_x.drop(del_list, axis=1)
    test_x_copy = test_x.drop(del_list, axis=1)

    return train_x_copy, test_x_copy

# [검증함수]

In [5]:
def find_best_threshold(y_true,y_prob,score_type):
    from sklearn.metrics import precision_recall_curve
    '''
    y실제값과 y예측값을 이용하여 score_type을 가장 높게하는 threshold를 찾는다
    '''
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    if score_type=="F1":
        f1_scores = 2*recall*precision/(recall+precision)
        max_f1_score = np.max(f1_scores)
    
        if thresholds[np.where(f1_scores == max_f1_score)]:
            max_threshold = thresholds[np.where(f1_scores == max_f1_score)][0]
            return max_threshold, max_f1_score
        else:
            return 0.5, -1

def see_feature_importance(model,train_x,test_x):
    '''
    input : model과 model을 학습한 train_x
    output : 선택한 중요도 이상의 columns들을 가진 train_x, test_x로 데이터 재세팅
    '''
    pd.set_option('display.max_rows', None)
    ft_importance_values = model.feature_importances_
    ft_series = pd.Series(ft_importance_values, index = train_x.columns)
    df_desc = pd.DataFrame(ft_series.sort_values(ascending=False)).reset_index()
    display(df_desc)
    
    # [그래프 그리기]
    df = pd.DataFrame(ft_series.sort_values(ascending=True)).reset_index()
    fig = plt.figure(figsize=(30,20))
    plt.barh(df["index"],df[0])
    
    # 값 그래프 옆에 보이기
    for i,v in enumerate(df[0]):
        plt.text(v,i,str(round(v,6)))
    plt.show()

    cut = float(input("몇 이상부터 넣을지?"))

    df_cutted = df_desc.loc[df_desc[0]>=cut]
    display(df_cutted)
    df_cutted_list = list(df_cutted["index"])

    train_x_copy = train_x.copy()
    test_x_copy = test_x.copy()

    for col in train_x.columns:
        if col not in df_cutted_list:
            train_x_copy = train_x_copy.drop([col],axis=1)
            test_x_copy = test_x_copy.drop([col],axis=1)

    return train_x_copy,test_x_copy

def grid_search_for_param_(model_name,scoring,cat_features=[]):
    '''
    모델과 score를 입력하면 최적파라미터를 찾아준다
    '''

    if model_name == "CatBoostClassifier":
        model = CatBoostClassifier()
        param_grid = {'depth': [5,10,15],
                      'learning_rate' : [0.01,0.05,0.1,],
                      'iterations'    : [300],
                      'cat_features' : [cat_features],
                      'early_stopping_rounds' : [50],
                    }

    elif model_name == "XGBClassifier":
        model = XGBClassifier()
        param_grid ={'max_depth' : [3,6,9] , 
                     'n_estimators': [300], # 트리의 개수, 높을수록 오래걸림
                     'learning_rate':[0.01, 0.1, 0.5], 
                     'gamma': [0, 1, 2], 
                     'random_state':[99]}

    elif model_name == "LGBMClassifier":
        model = LGBMClassifier()
        param_grid = {'num_leaves': [31, 127],
                      'reg_alpha' : [1,1.2], # L1 정규화 계수
                      'reg_lambda' : [1,1.2,1.4], # L2 정규화 계수
                      'min_data_in_leaf': [30, 50, 100, 300, 400],
                      'lambda_l1': [0, 1, 1.5],
                      'lambda_l2': [0, 1],
                      'learning_rate': [0.005, 0.01],
                      'n_estimators': [100,200,300], # 트리의 개수
                      'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
                      }

    gcv = GridSearchCV(estimator=model(),
                    param_grid = param_grid, 
                    scoring = scoring, 
                    cv = 3, # cross validation 몇개로 나눌것인지
                    n_jobs=-1, # 멀티프로세스 증가, 높을수록 빠르지만 cpu가 충분해야함)
                    refit=True
                    ).fit(train_x,train_y)

    display(gcv.cv_results_["params"])
    print(gcv.best_score_)
    print(gcv.best_params_)
    model = gcv.best_estimator_ # 모델

    # display(gcv.predict(test_x)) # 예측(분류)
    # display(gcv.predict_proba(test_x)) # 예측(확률)
    return gcv.best_score_,gcv.cv_results_["params"]

# [시각화 함수]

In [6]:
def draw_histogram(df,cols):
    '''
    cols : column이름의 리스트
    컬럼별 히스토그램을 그린다
    '''
    for col in cols:
        vc = dict(df[col].value_counts())

        plt.figure(figsize=(6,4))
        plt.hist(df[col], label=df[col].unique()) #label별 개수를 count해서 히스토그램을 그린다
        plt.xticks(rotation=90)
        plt.title(col)
        for domain,counts in vc.items():
            plt.text(domain,counts,counts) # (x,y,표시할값) X값이 x이고, Y값이 y인 지점에 '표시할값'을 표시한다
        plt.grid()  

def show_korean():
    '''
    한국어를 제대로 보이게 한다
    '''
    import matplotlib.pyplot as plt
    plt.rc('font', family='NanumBarunGothic') 
    
show_korean()

def draw_boxplot(df,cols):
    '''
    df : dataframe
    cols : column명이 적혀있는 list
    boxplot을 그린다
    '''
    for col in cols:
        plt.figure(figsize=(6,4))
        sns.boxplot(data=df[col],color="red")
        plt.title(col)
        plt.show()

def grapviz(hist):
    '''
    hist : 모델 fit return 값
    hist를 가지고 train, val set의 loss 및 accuracy를확인한다
    '''
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()
    
    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuracy')

    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')
    
    return plt.show()

In [7]:
bool_cols = ["d_l_match_yn","d_m_match_yn","d_s_match_yn","h_l_match_yn","h_m_match_yn","h_s_match_yn"]

nominal_cols = ["person_attribute_a","person_prefer_c","person_prefer_d_1","person_prefer_d_2","person_prefer_d_3",
                "person_prefer_f","person_prefer_g","person_prefer_h_1","person_prefer_h_2","person_prefer_h_3",
                "contents_attribute_i","contents_attribute_a","contents_attribute_j_1","contents_attribute_j",
                "contents_attribute_c","contents_attribute_k","contents_attribute_l","contents_attribute_d",
                "contents_attribute_m","contents_attribute_h"]

ordinal_cols = ["person_attribute_a_1","person_attribute_b",'person_prefer_e','contents_attribute_e']

# [전처리, 3이 best]

# 1. D,H,L 컬럼 삭제

In [ ]:
# data load
train = pd.read_csv(path+'/train.csv')
test_x = pd.read_csv(path+'/test.csv')

train_y = train[['target']]
train_x = train.drop(columns=['target'])

# 삭제
del_cols = ["id","person_prefer_d_1","person_prefer_d_2","person_prefer_d_3","contents_attribute_d",
            "person_prefer_h_1","person_prefer_h_2","person_prefer_h_3","contents_attribute_h",
            "contents_attribute_l","person_rn","contents_rn"]

train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)

# true, false => label encoding
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# nominal => onehot encoding
train_x,test_x = do_onehot_v2(train_x,test_x,list(set(nominal_cols)-set(del_cols)))

# scaling
train_x,test_x = do_scaling(train_x,test_x,ordinal_cols,"minmax")

# time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# freq one hot
train_x, test_x = convert_to_freq(train_x ,train_y, test_x, ["month","weekday","hour"])

display(train_x, test_x)

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a_1,person_attribute_b,person_prefer_e,contents_attribute_e,person_prefer_g:dummy:1,person_prefer_f:dummy:1,contents_attribute_a:dummy:1,contents_attribute_a:dummy:2,contents_attribute_a:dummy:3,person_prefer_c:dummy:1,person_prefer_c:dummy:2,person_prefer_c:dummy:3,person_prefer_c:dummy:4,person_prefer_c:dummy:5,contents_attribute_i:dummy:1,contents_attribute_i:dummy:2,contents_attribute_i:dummy:3,contents_attribute_c:dummy:1,contents_attribute_c:dummy:2,contents_attribute_c:dummy:3,contents_attribute_c:dummy:4,contents_attribute_m:dummy:1,contents_attribute_m:dummy:2,contents_attribute_m:dummy:3,contents_attribute_m:dummy:4,contents_attribute_m:dummy:5,contents_attribute_k:dummy:1,contents_attribute_k:dummy:2,contents_attribute_j_1:dummy:1,contents_attribute_j_1:dummy:2,contents_attribute_j_1:dummy:3,contents_attribute_j_1:dummy:4,contents_attribute_j_1:dummy:5,contents_attribute_j_1:dummy:6,contents_attribute_j_1:dummy:8,contents_attribute_j_1:dummy:9,contents_attribute_j_1:dummy:10,contents_attribute_j:dummy:1,contents_attribute_j:dummy:2,person_attribute_a:dummy:1,person_attribute_a:dummy:2,freq_onehot_month:1,freq_onehot_month:0,freq_onehot_weekday:1,freq_onehot_weekday:0,freq_onehot_hour:1,freq_onehot_hour:0
0,1,1,1,0,0,0,0.571429,0.6,0.727273,0.363636,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0.489504,0.510496,0.500963,0.499037,0.515023,0.484977
1,0,0,0,1,1,0,0.428571,0.8,0.363636,0.363636,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0.501433,0.498567,0.499121,0.500879,0.497675,0.502325
2,0,0,0,1,0,0,0.000000,0.6,0.272727,0.363636,1,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0.509325,0.490675,0.503222,0.496778,0.482290,0.517710
3,0,0,0,1,0,0,0.000000,0.4,0.272727,0.272727,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0.489504,0.510496,0.501556,0.498444,0.491195,0.508805
4,1,1,1,0,0,0,0.428571,0.8,0.363636,0.363636,1,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0.497787,0.502213,0.501556,0.498444,0.482290,0.517710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,0,0,0,1,0,0,0.142857,0.4,0.272727,0.454545,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0.497787,0.502213,0.500963,0.499037,0.515023,0.484977
501947,1,1,0,1,0,0,0.857143,0.4,0.363636,0.363636,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0.489504,0.510496,0.501556,0.498444,0.522432,0.477568
501948,1,1,1,1,0,0,1.000000,0.8,0.363636,0.636364,1,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0.503486,0.496514,0.503222,0.496778,0.497675,0.502325
501949,1,0,0,1,0,0,0.142857,0.4,0.363636,0.363636,1,1,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0.501433,0.498567,0.501556,0.498444,0.520038,0.479962


,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a_1,person_attribute_b,person_prefer_e,contents_attribute_e,person_prefer_g:dummy:1,person_prefer_f:dummy:1,contents_attribute_a:dummy:1,contents_attribute_a:dummy:2,contents_attribute_a:dummy:3,person_prefer_c:dummy:1,person_prefer_c:dummy:2,person_prefer_c:dummy:3,person_prefer_c:dummy:4,person_prefer_c:dummy:5,contents_attribute_i:dummy:1,contents_attribute_i:dummy:2,contents_attribute_i:dummy:3,contents_attribute_c:dummy:1,contents_attribute_c:dummy:2,contents_attribute_c:dummy:3,contents_attribute_c:dummy:4,contents_attribute_m:dummy:1,contents_attribute_m:dummy:2,contents_attribute_m:dummy:3,contents_attribute_m:dummy:4,contents_attribute_m:dummy:5,contents_attribute_k:dummy:1,contents_attribute_k:dummy:2,contents_attribute_j_1:dummy:1,contents_attribute_j_1:dummy:2,contents_attribute_j_1:dummy:3,contents_attribute_j_1:dummy:4,contents_attribute_j_1:dummy:5,contents_attribute_j_1:dummy:6,contents_attribute_j_1:dummy:8,contents_attribute_j_1:dummy:9,contents_attribute_j_1:dummy:10,contents_attribute_j:dummy:1,contents_attribute_j:dummy:2,person_attribute_a:dummy:1,person_attribute_a:dummy:2,freq_onehot_month:1,freq_onehot_month:0,freq_onehot_weekday:1,freq_onehot_weekday:0,freq_onehot_hour:1,freq_onehot_hour:0
0,1,0,0,1,1,1,0.142857,0.4,0.363636,0.454545,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0.49996,0.50004,0.498675,0.501325,0.441021,0.558979
1,0,0,0,1,0,0,0.000000,0.4,0.181818,0.363636,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0.49996,0.50004,0.499121,0.500879,0.452622,0.547378
2,1,0,0,1,1,1,0.428571,0.4,0.000000,0.363636,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0.49996,0.50004,0.499121,0.500879,0.462973,0.537027
3,1,0,0,1,1,1,0.285714,0.4,0.363636,0.272727,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0.49996,0.50004,0.499121,0.500879,0.484552,0.515448
4,1,0,0,1,0,0,0.857143,0.8,0.454545,0.363636,1,1,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0.49996,0.50004,0.500963,0.499037,0.475429,0.524571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,1,1,1,0,0,0,0.000000,0.8,0.363636,0.454545,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0.49996,0.50004,0.499121,0.500879,0.444165,0.555835
46400,1,0,0,1,0,0,0.000000,0.8,0.363636,0.363636,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0.49996,0.50004,0.498675,0.501325,0.475429,0.524571
46401,1,1,0,1,1,1,0.000000,0.6,0.454545,0.363636,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0.49996,0.50004,0.499121,0.500879,0.484552,0.515448
46402,1,1,0,1,1,1,0.428571,0.6,0.545455,0.454545,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0.49996,0.50004,0.501556,0.498444,0.518616,0.481384


# 2. 일치여부만 컬럼으로 (다같아서 그냥 삭제)

In [ ]:
def check_matching(train_x,test_x,col1,col2):
    train_x['is_matching:'+col1+"&"+col2] = train_x[col1]==train_x[col1]
    test_x['is_matching:'+col1+"&"+col2] = test_x[col1]==test_x[col1]

    train_x,test_x = do_label_encoding(train_x, test_x, ['is_matching:'+col1+"&"+col2])
    
    train_x = train_x.drop([col1,col2],axis=1)
    test_x = test_x.drop([col1,col2],axis=1)

    return train_x,test_x


# data load
train = pd.read_csv(path+'/train.csv')
test_x = pd.read_csv(path+'/test.csv')

train_y = train[['target']]
train_x = train.drop(columns=['target'])

# 삭제
del_cols = ["id","person_prefer_d_1","person_prefer_d_2","person_prefer_d_3","contents_attribute_d",
            "person_prefer_h_1","person_prefer_h_2","person_prefer_h_3","contents_attribute_h",
            "contents_attribute_l","person_rn","contents_rn",
            "person_attribute_a","contents_attribute_a",
            "person_prefer_c","contents_attribute_c",
            "person_prefer_e","contents_attribute_e"]

train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)

# true, false => label encoding
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# ordinal => onehot encoding
to_one_hot_list = []
train_x,test_x = do_onehot_v2(train_x,test_x,list(set(nominal_cols)-set(del_cols)))

# scaling
train_x,test_x = do_scaling(train_x,test_x,list(set(ordinal_cols)-set(del_cols)),"minmax")

# # time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# # freq one hot
train_x, test_x = convert_to_freq(train_x ,train_y, test_x, ["month","weekday","hour"])

display(train_x,test_x)

# 3. 속성 세분화 및 일치여부 컬럼

a : 각컬럼,일치  
a_1 : 스캐일링  
b : 스캐일링  
c : 컬럼,일치  
d : 세분화일치  
e : 각컬럼, 일치  
f : 각컬럼  
g : 각컬럼  
h : 세분화일치  
i : 각컬럼  
j : 각컬럼  
k : 각컬럼  
l : 각컬럼(도메인 너무많음)  
m : 각컬럼  

In [8]:
# [data load]
train = pd.read_csv(path+'/train.csv')
train_y = train[['target']]
train_x = train.drop(columns=['target'])

test_x = pd.read_csv(path+'/test.csv')

# [속성 세부 분류로 분해] 
def to_detail(train_x, test_x, x_code_col, detail_code_df, detail_code_col):
    '''
    x의 컬럼 x_code_col을 detail_code_df로 분해한다
    '''
    train_x = pd.merge(train_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    test_x = pd.merge(test_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    
    for col in list(detail_code_df.columns):
        train_x = train_x.rename(columns={col:col+":"+x_code_col})
        test_x = test_x.rename(columns={col:col+":"+x_code_col})

    train_x = train_x.drop([x_code_col],axis=1)
    test_x = test_x.drop([x_code_col],axis=1)

    return train_x, test_x

# 속성 D
att_D_code = pd.read_csv(path+"/속성_D_코드.csv").drop(["Unnamed: 5"],axis=1)
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_1", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_2", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_3", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_d", att_D_code, "속성 D 코드")

# 속성 H
att_H_code = pd.read_csv(path+"/속성_H_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_1", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_2", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_3", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_h", att_H_code, "속성 H 코드")

# 속성 L
att_L_code = pd.read_csv(path+"/속성_L_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_l", att_L_code, "속성 L 코드")


# [일치여부 칼럼 추가]
def equal_col(train_x,test_x,person_col,contents_col):
    cols = train_x.columns

    train_x_copy = train_x.copy()
    test_x_copy = test_x.copy()

    for col in cols:
        if ":" not in col:
            continue

        a,b = col.split(":")

        if b==person_col:
            train_x_copy["is_equl#"+col] = train_x_copy[col] == train_x_copy[a+":"+contents_col]
            test_x_copy["is_equl#"+col] = test_x_copy[col] == test_x_copy[a+":"+contents_col]

    return train_x_copy,test_x_copy

# 속성 D
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_1","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_2","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_3","contents_attribute_d")
# 속성 H
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_1","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_2","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_3","contents_attribute_h")
# 속성 a
train_x["is_equl#person_attribute_a"] = train_x["person_attribute_a"] == train_x["contents_attribute_a"]
test_x["is_equl#person_attribute_a"] = test_x["person_attribute_a"] == test_x["contents_attribute_a"]
# 속성 c
train_x["is_equl#person_prefer_c"] = train_x["person_prefer_c"] == train_x["contents_attribute_c"]
test_x["is_equl#person_prefer_c"] = test_x["person_prefer_c"] == test_x["contents_attribute_c"]
# 속성 e
train_x["is_equl#person_prefer_e"] = train_x["person_prefer_e"] == train_x["contents_attribute_e"]
test_x["is_equl#person_prefer_e"] = test_x["person_prefer_e"] == test_x["contents_attribute_e"]

# [삭제]
del_cols = ["id","d_l_match_yn","d_m_match_yn","d_s_match_yn","h_l_match_yn","h_m_match_yn","h_s_match_yn","person_rn","contents_rn"]
train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)


# bool column labeling
bool_cols = list(train_x.select_dtypes(bool).columns)
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# one_hot
to_onehot_list = ["contents_attribute_a","contents_attribute_c",
                  "person_attribute_a","person_prefer_c","person_prefer_f","person_prefer_g",
                  "contents_attribute_i","contents_attribute_j_1","contents_attribute_j",
                  "contents_attribute_k","contents_attribute_m"]
train_x, test_x = do_onehot_v2(train_x, test_x, to_onehot_list)

# time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# scaling
train_x,test_x = do_scaling(train_x, test_x, ordinal_cols, "minmax")

# cat feature
categorical_feature = list(set(train_x.columns)-set(ordinal_cols))

print(categorical_feature)
display(train_x,test_x)

['속성 D 소분류코드:contents_attribute_d', '속성 D 소분류코드:person_prefer_d_2', 'contents_attribute_i:dummy:2', 'contents_attribute_c:dummy:2', '속성 L 대분류코드:contents_attribute_l', 'contents_attribute_j_1:dummy:8', 'contents_attribute_j_1:dummy:2', 'is_equl#속성 H 코드:person_prefer_h_3', 'contents_attribute_c:dummy:1', 'contents_attribute_k:dummy:2', 'is_equl#속성 H 상위코드:person_prefer_h_3', '속성 H 코드:person_prefer_h_3', 'is_equl#속성 D 소분류코드:person_prefer_d_2', 'is_equl#속성 D 세분류코드:person_prefer_d_3', 'contents_attribute_i:dummy:3', 'month', 'is_equl#속성 H 코드:person_prefer_h_2', '속성 D 코드:person_prefer_d_3', 'contents_attribute_j_1:dummy:10', 'contents_attribute_m:dummy:5', '속성 L 세분류코드:contents_attribute_l', 'person_prefer_c:dummy:3', 'is_equl#속성 D 대분류코드:person_prefer_d_3', '속성 L 코드:contents_attribute_l', '속성 D 세분류코드:person_prefer_d_3', 'contents_attribute_a:dummy:3', 'contents_attribute_a:dummy:1', 'is_equl#속성 D 중분류코드:person_prefer_d_1', '속성 D 세분류코드:person_prefer_d_1', 'is_equl#속성 H 상위코드:person_prefer_h_2', '

,person_attribute_a_1,person_attribute_b,person_prefer_e,contents_attribute_e,속성 D 코드:person_prefer_d_1,속성 D 세분류코드:person_prefer_d_1,속성 D 소분류코드:person_prefer_d_1,속성 D 중분류코드:person_prefer_d_1,속성 D 대분류코드:person_prefer_d_1,속성 D 코드:person_prefer_d_2,속성 D 세분류코드:person_prefer_d_2,속성 D 소분류코드:person_prefer_d_2,속성 D 중분류코드:person_prefer_d_2,속성 D 대분류코드:person_prefer_d_2,속성 D 코드:person_prefer_d_3,속성 D 세분류코드:person_prefer_d_3,속성 D 소분류코드:person_prefer_d_3,속성 D 중분류코드:person_prefer_d_3,속성 D 대분류코드:person_prefer_d_3,속성 D 코드:contents_attribute_d,속성 D 세분류코드:contents_attribute_d,속성 D 소분류코드:contents_attribute_d,속성 D 중분류코드:contents_attribute_d,속성 D 대분류코드:contents_attribute_d,속성 H 코드:person_prefer_h_1,속성 H 상위코드:person_prefer_h_1,속성 H 코드:person_prefer_h_2,속성 H 상위코드:person_prefer_h_2,속성 H 코드:person_prefer_h_3,속성 H 상위코드:person_prefer_h_3,속성 H 코드:contents_attribute_h,속성 H 상위코드:contents_attribute_h,속성 L 코드:contents_attribute_l,속성 L 세분류코드:contents_attribute_l,속성 L 소분류코드:contents_attribute_l,속성 L 중분류코드:contents_attribute_l,속성 L 대분류코드:contents_attribute_l,is_equl#속성 D 코드:person_prefer_d_1,is_equl#속성 D 세분류코드:person_prefer_d_1,is_equl#속성 D 소분류코드:person_prefer_d_1,...,contents_attribute_a:dummy:1,contents_attribute_a:dummy:2,contents_attribute_a:dummy:3,contents_attribute_c:dummy:1,contents_attribute_c:dummy:2,contents_attribute_c:dummy:3,contents_attribute_c:dummy:4,person_attribute_a:dummy:1,person_attribute_a:dummy:2,person_prefer_c:dummy:1,person_prefer_c:dummy:2,person_prefer_c:dummy:3,person_prefer_c:dummy:4,person_prefer_c:dummy:5,person_prefer_f:dummy:1,person_prefer_g:dummy:1,contents_attribute_i:dummy:1,contents_attribute_i:dummy:2,contents_attribute_i:dummy:3,contents_attribute_j_1:dummy:1,contents_attribute_j_1:dummy:2,contents_attribute_j_1:dummy:3,contents_attribute_j_1:dummy:4,contents_attribute_j_1:dummy:5,contents_attribute_j_1:dummy:6,contents_attribute_j_1:dummy:8,contents_attribute_j_1:dummy:9,contents_attribute_j_1:dummy:10,contents_attribute_j:dummy:1,contents_attribute_j:dummy:2,contents_attribute_k:dummy:1,contents_attribute_k:dummy:2,contents_attribute_m:dummy:1,contents_attribute_m:dummy:2,contents_attribute_m:dummy:3,contents_attribute_m:dummy:4,contents_attribute_m:dummy:5,month,weekday,hour
0,0.571429,0.6,0.727273,0.363636,275,275,274,274,216,370,369,297,368,216,369,369,297,368,216,275,275,274,274,216,4,3,95,94,59,58,139,94,1608,1607,1606,1605,2016,1,1,1,...,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,4,12
1,0.428571,0.8,0.363636,0.363636,114,114,56,109,1,181,175,56,152,1,175,175,56,152,1,275,275,274,274,216,131,94,101,94,96,94,133,94,1608,1607,1606,1605,2016,0,0,0,...,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,6,3,17
2,0.000000,0.6,0.272727,0.363636,464,464,450,463,377,175,175,56,152,1,452,452,450,451,377,94,92,56,91,1,54,48,263,250,56,48,53,48,1600,1599,1595,1572,2016,0,0,0,...,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,7,2,20
3,0.000000,0.4,0.272727,0.272727,703,703,690,703,618,705,703,690,703,618,704,703,690,703,618,275,275,274,274,216,72,71,227,226,2,1,74,71,1608,1607,1606,1605,2016,0,0,0,...,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,18
4,0.428571,0.8,0.363636,0.363636,275,275,274,274,216,370,369,297,368,216,369,369,297,368,216,275,275,274,274,216,214,208,210,208,209,208,74,71,1608,1607,1606,1605,2016,1,1,1,...,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,3,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,0.142857,0.4,0.272727,0.454545,1192,1192,1169,1180,926,935,929,927,928,926,1228,1227,1227,1227,926,147,145,56,123,1,59,58,4,3,95,94,65,58,354,352,347,346,2006,0,0,0,...,0,0,1,1,0,0,0,1

,person_attribute_a_1,person_attribute_b,person_prefer_e,contents_attribute_e,속성 D 코드:person_prefer_d_1,속성 D 세분류코드:person_prefer_d_1,속성 D 소분류코드:person_prefer_d_1,속성 D 중분류코드:person_prefer_d_1,속성 D 대분류코드:person_prefer_d_1,속성 D 코드:person_prefer_d_2,속성 D 세분류코드:person_prefer_d_2,속성 D 소분류코드:person_prefer_d_2,속성 D 중분류코드:person_prefer_d_2,속성 D 대분류코드:person_prefer_d_2,속성 D 코드:person_prefer_d_3,속성 D 세분류코드:person_prefer_d_3,속성 D 소분류코드:person_prefer_d_3,속성 D 중분류코드:person_prefer_d_3,속성 D 대분류코드:person_prefer_d_3,속성 D 코드:contents_attribute_d,속성 D 세분류코드:contents_attribute_d,속성 D 소분류코드:contents_attribute_d,속성 D 중분류코드:contents_attribute_d,속성 D 대분류코드:contents_attribute_d,속성 H 코드:person_prefer_h_1,속성 H 상위코드:person_prefer_h_1,속성 H 코드:person_prefer_h_2,속성 H 상위코드:person_prefer_h_2,속성 H 코드:person_prefer_h_3,속성 H 상위코드:person_prefer_h_3,속성 H 코드:contents_attribute_h,속성 H 상위코드:contents_attribute_h,속성 L 코드:contents_attribute_l,속성 L 세분류코드:contents_attribute_l,속성 L 소분류코드:contents_attribute_l,속성 L 중분류코드:contents_attribute_l,속성 L 대분류코드:contents_attribute_l,is_equl#속성 D 코드:person_prefer_d_1,is_equl#속성 D 세분류코드:person_prefer_d_1,is_equl#속성 D 소분류코드:person_prefer_d_1,...,contents_attribute_a:dummy:1,contents_attribute_a:dummy:2,contents_attribute_a:dummy:3,contents_attribute_c:dummy:1,contents_attribute_c:dummy:2,contents_attribute_c:dummy:3,contents_attribute_c:dummy:4,person_attribute_a:dummy:1,person_attribute_a:dummy:2,person_prefer_c:dummy:1,person_prefer_c:dummy:2,person_prefer_c:dummy:3,person_prefer_c:dummy:4,person_prefer_c:dummy:5,person_prefer_f:dummy:1,person_prefer_g:dummy:1,contents_attribute_i:dummy:1,contents_attribute_i:dummy:2,contents_attribute_i:dummy:3,contents_attribute_j_1:dummy:1,contents_attribute_j_1:dummy:2,contents_attribute_j_1:dummy:3,contents_attribute_j_1:dummy:4,contents_attribute_j_1:dummy:5,contents_attribute_j_1:dummy:6,contents_attribute_j_1:dummy:8,contents_attribute_j_1:dummy:9,contents_attribute_j_1:dummy:10,contents_attribute_j:dummy:1,contents_attribute_j:dummy:2,contents_attribute_k:dummy:1,contents_attribute_k:dummy:2,contents_attribute_m:dummy:1,contents_attribute_m:dummy:2,contents_attribute_m:dummy:3,contents_attribute_m:dummy:4,contents_attribute_m:dummy:5,month,weekday,hour
0,0.142857,0.4,0.363636,0.454545,857,854,824,853,744,851,847,824,847,744,1227,1227,1227,1227,926,839,836,824,831,744,263,250,56,48,49,48,263,250,1147,1146,1128,1021,2010,0,0,1,...,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,12,1,2
1,0.000000,0.4,0.181818,0.363636,683,683,659,677,618,1086,1086,1053,1086,926,662,662,659,660,618,278,276,274,274,216,258,250,263,250,49,48,263,250,1611,1610,1606,1605,2016,0,0,0,...,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,12,3,5
2,0.428571,0.4,0.000000,0.363636,514,514,482,509,482,790,784,745,783,744,1233,1227,1227,1227,926,490,490,482,490,482,177,169,170,169,171,169,177,169,1817,1812,1811,1810,2020,0,0,1,...,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,12,3,23
3,0.285714,0.4,0.363636,0.272727,114,114,56,109,1,181,175,56,152,1,175,175,56,152,1,150,145,56,123,1,177,169,170,169,171,169,177,169,101,101,100,99,2006,0,0,1,...,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,12,3,19
4,0.857143,0.8,0.454545,0.363636,1082,1078,1053,1078,926,1078,1078,1053,1078,926,1056,1056,1053,1054,926,1097,1097,1093,1094,926,178,169,177,169,4,3,177,169,985,984,980,954,2009,0,0,0,...,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,12,4,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,0.000000,0.8,0.363636,0.454545,147,145,56,123,1,46,46,2,43,1,145,145,56,123,1,147,145,56,123,1,2,1,4,3,95,94,91,85,759,759,759,759,2006,1

# 4
(1) person, content 추가

In [ ]:
# [data load]
train = pd.read_csv(path+'/train.csv')
train_y = train[['target']]
train_x = train.drop(columns=['target'])

test_x = pd.read_csv(path+'/test.csv')

# [속성 세부 분류로 분해] 
def to_detail(train_x, test_x, x_code_col, detail_code_df, detail_code_col):
    '''
    x의 컬럼 x_code_col을 detail_code_df로 분해한다
    '''
    train_x = pd.merge(train_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    test_x = pd.merge(test_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    
    for col in list(detail_code_df.columns):
        train_x = train_x.rename(columns={col:col+":"+x_code_col})
        test_x = test_x.rename(columns={col:col+":"+x_code_col})

    train_x = train_x.drop([x_code_col],axis=1)
    test_x = test_x.drop([x_code_col],axis=1)

    return train_x, test_x

# 속성 D
att_D_code = pd.read_csv(path+"/속성_D_코드.csv").drop(["Unnamed: 5"],axis=1)
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_1", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_2", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_3", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_d", att_D_code, "속성 D 코드")

# 속성 H
att_H_code = pd.read_csv(path+"/속성_H_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_1", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_2", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_3", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_h", att_H_code, "속성 H 코드")

# 속성 L
att_L_code = pd.read_csv(path+"/속성_L_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_l", att_L_code, "속성 L 코드")


# [일치여부 칼럼 추가]
def equal_col(train_x,test_x,person_col,contents_col):
    cols = train_x.columns

    train_x_copy = train_x.copy()
    test_x_copy = test_x.copy()

    for col in cols:
        if ":" not in col:
            continue

        a,b = col.split(":")

        if b==person_col:
            train_x_copy["is_equl#"+col] = train_x_copy[col] == train_x_copy[a+":"+contents_col]
            test_x_copy["is_equl#"+col] = test_x_copy[col] == test_x_copy[a+":"+contents_col]

    return train_x_copy,test_x_copy

# 속성 D
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_1","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_2","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_3","contents_attribute_d")
# 속성 H
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_1","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_2","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_3","contents_attribute_h")
# 속성 a
train_x["is_equl#person_attribute_a"] = train_x["person_attribute_a"] == train_x["contents_attribute_a"]
test_x["is_equl#person_attribute_a"] = test_x["person_attribute_a"] == test_x["contents_attribute_a"]
# 속성 c
train_x["is_equl#person_prefer_c"] = train_x["person_prefer_c"] == train_x["contents_attribute_c"]
test_x["is_equl#person_prefer_c"] = test_x["person_prefer_c"] == test_x["contents_attribute_c"]
# 속성 e
train_x["is_equl#person_prefer_e"] = train_x["person_prefer_e"] == train_x["contents_attribute_e"]
test_x["is_equl#person_prefer_e"] = test_x["person_prefer_e"] == test_x["contents_attribute_e"]

# [삭제]
del_cols = ["id","d_l_match_yn","d_m_match_yn","d_s_match_yn","h_l_match_yn","h_m_match_yn","h_s_match_yn"]
train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)


# bool column labeling
bool_cols = list(train_x.select_dtypes(bool).columns)
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# one_hot
to_onehot_list = ["contents_attribute_a","contents_attribute_c",
                  "person_attribute_a","person_prefer_c","person_prefer_f","person_prefer_g",
                  "contents_attribute_i","contents_attribute_j_1","contents_attribute_j",
                  "contents_attribute_k","contents_attribute_m"]
train_x, test_x = do_onehot_v2(train_x, test_x, to_onehot_list)

# time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# scaling
train_x,test_x = do_scaling(train_x, test_x, ordinal_cols, "minmax")

# cat feature
categorical_feature = list(set(train_x.columns)-set(ordinal_cols))

print(categorical_feature)
display(train_x,test_x)

# 5. person, content추가, 일치여부컬럼만 남기기


In [ ]:
# [data load]
train = pd.read_csv(path+'/train.csv')
train_y = train[['target']]
train_x = train.drop(columns=['target'])

test_x = pd.read_csv(path+'/test.csv')

# [속성 세부 분류로 분해] 
def to_detail(train_x, test_x, x_code_col, detail_code_df, detail_code_col):
    '''
    x의 컬럼 x_code_col을 detail_code_df로 분해한다
    '''
    train_x = pd.merge(train_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    test_x = pd.merge(test_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    
    for col in list(detail_code_df.columns):
        train_x = train_x.rename(columns={col:col+":"+x_code_col})
        test_x = test_x.rename(columns={col:col+":"+x_code_col})

    train_x = train_x.drop([x_code_col],axis=1)
    test_x = test_x.drop([x_code_col],axis=1)

    return train_x, test_x

# 속성 D
att_D_code = pd.read_csv(path+"/속성_D_코드.csv").drop(["Unnamed: 5"],axis=1)
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_1", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_2", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_3", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_d", att_D_code, "속성 D 코드")

# 속성 H
att_H_code = pd.read_csv(path+"/속성_H_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_1", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_2", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_3", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_h", att_H_code, "속성 H 코드")

# 속성 L
att_L_code = pd.read_csv(path+"/속성_L_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_l", att_L_code, "속성 L 코드")


# [일치여부 칼럼 추가]
def equal_col(train_x,test_x,person_col,contents_col):
    cols = train_x.columns

    train_x_copy = train_x.copy()
    test_x_copy = test_x.copy()

    for col in cols:
        if ":" not in col:
            continue

        a,b = col.split(":")

        if b==person_col:
            train_x_copy["is_equl#"+col] = train_x_copy[col] == train_x_copy[a+":"+contents_col]
            test_x_copy["is_equl#"+col] = test_x_copy[col] == test_x_copy[a+":"+contents_col]

    return train_x_copy,test_x_copy

# 속성 D
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_1","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_2","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_3","contents_attribute_d")
# 속성 H
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_1","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_2","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_3","contents_attribute_h")
# 속성 a
train_x["is_equl#person_attribute_a"] = train_x["person_attribute_a"] == train_x["contents_attribute_a"]
test_x["is_equl#person_attribute_a"] = test_x["person_attribute_a"] == test_x["contents_attribute_a"]
# 속성 c
train_x["is_equl#person_prefer_c"] = train_x["person_prefer_c"] == train_x["contents_attribute_c"]
test_x["is_equl#person_prefer_c"] = test_x["person_prefer_c"] == test_x["contents_attribute_c"]
# 속성 e
train_x["is_equl#person_prefer_e"] = train_x["person_prefer_e"] == train_x["contents_attribute_e"]
test_x["is_equl#person_prefer_e"] = test_x["person_prefer_e"] == test_x["contents_attribute_e"]

# [삭제]
del_cols = ["id","d_l_match_yn","d_m_match_yn","d_s_match_yn","h_l_match_yn","h_m_match_yn","h_s_match_yn",
            '속성 D 코드:person_prefer_d_1', '속성 D 세분류코드:person_prefer_d_1',
            '속성 D 소분류코드:person_prefer_d_1', '속성 D 중분류코드:person_prefer_d_1',
            '속성 D 대분류코드:person_prefer_d_1', '속성 D 코드:person_prefer_d_2',
            '속성 D 세분류코드:person_prefer_d_2', '속성 D 소분류코드:person_prefer_d_2',
            '속성 D 중분류코드:person_prefer_d_2', '속성 D 대분류코드:person_prefer_d_2',
            '속성 D 코드:person_prefer_d_3', '속성 D 세분류코드:person_prefer_d_3',
            '속성 D 소분류코드:person_prefer_d_3', '속성 D 중분류코드:person_prefer_d_3',
            '속성 D 대분류코드:person_prefer_d_3', '속성 D 코드:contents_attribute_d',
            '속성 D 세분류코드:contents_attribute_d', '속성 D 소분류코드:contents_attribute_d',
            '속성 D 중분류코드:contents_attribute_d', '속성 D 대분류코드:contents_attribute_d',
            '속성 H 코드:person_prefer_h_1', '속성 H 상위코드:person_prefer_h_1',
            '속성 H 코드:person_prefer_h_2', '속성 H 상위코드:person_prefer_h_2',
            '속성 H 코드:person_prefer_h_3', '속성 H 상위코드:person_prefer_h_3',
            '속성 H 코드:contents_attribute_h', '속성 H 상위코드:contents_attribute_h',]
train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)


# bool column labeling
bool_cols = list(train_x.select_dtypes(bool).columns)
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# one_hot
to_onehot_list = ["person_prefer_f","person_prefer_g","contents_attribute_i","contents_attribute_j_1",
                  "contents_attribute_j","contents_attribute_k","contents_attribute_m"]
train_x, test_x = do_onehot_v2(train_x, test_x, to_onehot_list)

# time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# scaling
ordinal_cols = list(set(ordinal_cols)-set(del_cols))
train_x,test_x = do_scaling(train_x, test_x, ordinal_cols, "minmax")

# cat feature
categorical_feature = list(set(train_x.columns)-set(ordinal_cols))

print(categorical_feature)
display(train_x,test_x)

# 6 원핫코딩 x

In [ ]:
# [data load]
train = pd.read_csv(path+'/train.csv')
train_y = train[['target']]
train_x = train.drop(columns=['target'])

test_x = pd.read_csv(path+'/test.csv')

# [속성 세부 분류로 분해] 
def to_detail(train_x, test_x, x_code_col, detail_code_df, detail_code_col):
    '''
    x의 컬럼 x_code_col을 detail_code_df로 분해한다
    '''
    train_x = pd.merge(train_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    test_x = pd.merge(test_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    
    for col in list(detail_code_df.columns):
        train_x = train_x.rename(columns={col:col+":"+x_code_col})
        test_x = test_x.rename(columns={col:col+":"+x_code_col})

    train_x = train_x.drop([x_code_col],axis=1)
    test_x = test_x.drop([x_code_col],axis=1)

    return train_x, test_x

# 속성 D
att_D_code = pd.read_csv(path+"/속성_D_코드.csv").drop(["Unnamed: 5"],axis=1)
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_1", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_2", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_3", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_d", att_D_code, "속성 D 코드")

# 속성 H
att_H_code = pd.read_csv(path+"/속성_H_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_1", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_2", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_3", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_h", att_H_code, "속성 H 코드")

# 속성 L
att_L_code = pd.read_csv(path+"/속성_L_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_l", att_L_code, "속성 L 코드")


# [일치여부 칼럼 추가]
def equal_col(train_x,test_x,person_col,contents_col):
    cols = train_x.columns

    train_x_copy = train_x.copy()
    test_x_copy = test_x.copy()

    for col in cols:
        if ":" not in col:
            continue

        a,b = col.split(":")

        if b==person_col:
            train_x_copy["is_equl#"+col] = train_x_copy[col] == train_x_copy[a+":"+contents_col]
            test_x_copy["is_equl#"+col] = test_x_copy[col] == test_x_copy[a+":"+contents_col]

    return train_x_copy,test_x_copy

# 속성 D
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_1","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_2","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_3","contents_attribute_d")
# 속성 H
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_1","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_2","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_3","contents_attribute_h")
# 속성 a
train_x["is_equl#person_attribute_a"] = train_x["person_attribute_a"] == train_x["contents_attribute_a"]
test_x["is_equl#person_attribute_a"] = test_x["person_attribute_a"] == test_x["contents_attribute_a"]
# 속성 c
train_x["is_equl#person_prefer_c"] = train_x["person_prefer_c"] == train_x["contents_attribute_c"]
test_x["is_equl#person_prefer_c"] = test_x["person_prefer_c"] == test_x["contents_attribute_c"]
# 속성 e
train_x["is_equl#person_prefer_e"] = train_x["person_prefer_e"] == train_x["contents_attribute_e"]
test_x["is_equl#person_prefer_e"] = test_x["person_prefer_e"] == test_x["contents_attribute_e"]

# [삭제]
del_cols = ["id","d_l_match_yn","d_m_match_yn","d_s_match_yn","h_l_match_yn","h_m_match_yn","h_s_match_yn","person_rn","contents_rn"]
train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)


# bool column labeling
bool_cols = list(train_x.select_dtypes(bool).columns)
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# # one_hot
# to_onehot_list = ["contents_attribute_a","contents_attribute_c",
#                   "person_attribute_a","person_prefer_c","person_prefer_f","person_prefer_g",
#                   "contents_attribute_i","contents_attribute_j_1","contents_attribute_j",
#                   "contents_attribute_k","contents_attribute_m"]
# train_x, test_x = do_onehot_v2(train_x, test_x, to_onehot_list)

# time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# scaling
train_x,test_x = do_scaling(train_x, test_x, ordinal_cols, "minmax")

# cat feature
categorical_feature = list(set(train_x.columns)-set(ordinal_cols))

print(categorical_feature)
display(train_x,test_x)

# 7 도메인이 하나거나 비율이 낮은 컬럼 삭제

In [ ]:
# [data load]
train = pd.read_csv(path+'/train.csv')
train_y = train[['target']]
train_x = train.drop(columns=['target'])

test_x = pd.read_csv(path+'/test.csv')

# [속성 세부 분류로 분해] 
def to_detail(train_x, test_x, x_code_col, detail_code_df, detail_code_col):
    '''
    x의 컬럼 x_code_col을 detail_code_df로 분해한다
    '''
    train_x = pd.merge(train_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    test_x = pd.merge(test_x,detail_code_df,how='left',left_on=x_code_col,right_on=detail_code_col)
    
    for col in list(detail_code_df.columns):
        train_x = train_x.rename(columns={col:col+":"+x_code_col})
        test_x = test_x.rename(columns={col:col+":"+x_code_col})

    train_x = train_x.drop([x_code_col],axis=1)
    test_x = test_x.drop([x_code_col],axis=1)

    return train_x, test_x

# 속성 D
att_D_code = pd.read_csv(path+"/속성_D_코드.csv").drop(["Unnamed: 5"],axis=1)
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_1", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_2", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_d_3", att_D_code, "속성 D 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_d", att_D_code, "속성 D 코드")

# 속성 H
att_H_code = pd.read_csv(path+"/속성_H_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_1", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_2", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "person_prefer_h_3", att_H_code, "속성 H 코드")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_h", att_H_code, "속성 H 코드")

# 속성 L
att_L_code = pd.read_csv(path+"/속성_L_코드.csv")
train_x, test_x = to_detail(train_x, test_x, "contents_attribute_l", att_L_code, "속성 L 코드")


# [일치여부 칼럼 추가]
def equal_col(train_x,test_x,person_col,contents_col):
    cols = train_x.columns

    train_x_copy = train_x.copy()
    test_x_copy = test_x.copy()

    for col in cols:
        if ":" not in col:
            continue

        a,b = col.split(":")

        if b==person_col:
            train_x_copy["is_equl#"+col] = train_x_copy[col] == train_x_copy[a+":"+contents_col]
            test_x_copy["is_equl#"+col] = test_x_copy[col] == test_x_copy[a+":"+contents_col]

    return train_x_copy,test_x_copy

# 속성 D
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_1","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_2","contents_attribute_d")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_d_3","contents_attribute_d")
# 속성 H
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_1","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_2","contents_attribute_h")
train_x, test_x = equal_col(train_x,test_x,"person_prefer_h_3","contents_attribute_h")
# 속성 a
train_x["is_equl#person_attribute_a"] = train_x["person_attribute_a"] == train_x["contents_attribute_a"]
test_x["is_equl#person_attribute_a"] = test_x["person_attribute_a"] == test_x["contents_attribute_a"]
# 속성 c
train_x["is_equl#person_prefer_c"] = train_x["person_prefer_c"] == train_x["contents_attribute_c"]
test_x["is_equl#person_prefer_c"] = test_x["person_prefer_c"] == test_x["contents_attribute_c"]
# 속성 e
train_x["is_equl#person_prefer_e"] = train_x["person_prefer_e"] == train_x["contents_attribute_e"]
test_x["is_equl#person_prefer_e"] = test_x["person_prefer_e"] == test_x["contents_attribute_e"]

# bool column labeling
bool_cols = list(train_x.select_dtypes(bool).columns)
train_x,test_x = do_label_encoding(train_x, test_x, bool_cols)

# one_hot
to_onehot_list = ["contents_attribute_a","contents_attribute_c",
                  "person_attribute_a","person_prefer_c","person_prefer_f","person_prefer_g",
                  "contents_attribute_i","contents_attribute_j_1","contents_attribute_j",
                  "contents_attribute_k","contents_attribute_m"]
train_x, test_x = do_onehot_v2(train_x, test_x, to_onehot_list)

# time_labeling
train_x = time_labeling(train_x,"contents_open_dt",["month","weekday","hour"])
test_x = time_labeling(test_x,"contents_open_dt",["month","weekday","hour"])

# scaling
train_x,test_x = do_scaling(train_x, test_x, ordinal_cols, "minmax")

# [삭제]
del_cols = ["id","d_l_match_yn","d_m_match_yn","d_s_match_yn","h_l_match_yn","h_m_match_yn","h_s_match_yn","person_rn","contents_rn",
            "contents_attribute_c:dummy:4","person_prefer_f:dummy:1","person_prefer_g:dummy:1","contents_attribute_j_1:dummy:8","contents_attribute_j_1:dummy:9"]
train_x = train_x.drop(columns=del_cols, axis=1)
test_x = test_x.drop(columns=del_cols, axis=1)

train_x, test_x = del_one_domain(train_x,test_x)

# cat feature
categorical_feature = list(set(train_x.columns)-set(ordinal_cols))

print(categorical_feature)
display(train_x,test_x)

# [모델 학습]

In [ ]:
def split_train(train_x, train_y, test_x, model_name, n_splits=4, n_estimator=200, is_stratified=True, eval_metric="F1",early_stopping_rounds=50, 
             categorical_feature=[], random_state=42, verbose=1, test_size=0.3):
    '''
    lgbm, xgb, catboost 모델을 train,val셋으로 나누어 학습한다
    '''

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score

    # [data, func setting]
    def custom_f1_LGBMClassifier(y_true, y_pred):
        y_pred = np.where(y_pred >= 0.5 , 1, 0)
        score = f1_score(y_true,y_pred)
        return ("f1_score", score, True)

    def custom_f1_XGBClassifier(y_pred, dtrain):
        y_true = np.array(list(map(int,dtrain.get_label())))
        y_pred = np.where(y_pred >= 0.5 , 1, 0)
        score = f1_score(y_true,y_pred)
        return ("f1_score_xgboost", score)

    eval_metric_dict_LGBMClassifier = {"F1":custom_f1_LGBMClassifier}
    eval_metric_dict_XGBClassifier = {"F1":custom_f1_XGBClassifier}
    eval_metric_dict_CatBoostClassifier = {"F1":"F1"}

    # [data split]
    split_train_x, split_val_x, split_train_y, split_val_y = train_test_split(train_x, train_y, test_size=test_size, shuffle=True)

    # [모델 선택]
    if model_name=="LGBMClassifier":
        model = LGBMClassifier(n_estimators=n_estimator, 
                               random_state=random_state)
        model.fit(split_train_x,split_train_y,
                  eval_set=[(split_val_x,split_val_y)],
                  early_stopping_rounds=early_stopping_rounds,
                  eval_metric=eval_metric_dict_LGBMClassifier[eval_metric],
                  verbose=verbose,
                  categorical_feature=categorical_feature,
                  )
    elif model_name=="XGBClassifier":
        model = XGBClassifier(n_estimators=n_estimator, 
                              random_state=random_state)
        model.fit(split_train_x,split_train_y,
                  eval_set=[(split_val_x,split_val_y)],
                  early_stopping_rounds=early_stopping_rounds,
                  eval_metric=eval_metric_dict_XGBClassifier[eval_metric],
                  verbose=verbose,
                  )
    elif model_name=="CatBoostClassifier":
        model = CatBoostClassifier(n_estimators=n_estimator, 
                                   random_state=random_state,
                                   eval_metric=eval_metric_dict_CatBoostClassifier[eval_metric], # catboost는 평가지표가 많다 
                                   cat_features=categorical_feature, # catboost는 이름이 "cat_features"
                                   one_hot_max_size=4)
        model.fit(split_train_x,split_train_y,
                  eval_set=[(split_val_x,split_val_y)],
                  early_stopping_rounds=early_stopping_rounds,
                  verbose=verbose,
                  )

    # [find best threshold]
    y_prob = np.array(model.predict_proba(split_val_x)[:, 1])
    y_true = np.array(split_val_y.iloc[:,0])
    
    best_threshold, best_score = find_best_threshold(y_true, y_prob, eval_metric)
    print("best score",best_score,"best_threshold",best_threshold)

    test_y_prob = np.array(model.predict_proba(test_x)[:, 1])
    test_y = np.where(test_y_prob >= best_threshold , 1, 0)
    
    return model, test_y

# params = {"model_name" : "LGBMClassifier",
#           "n_splits" : 3, # cross vali 몇개로 나눌것인가
#           "n_estimator" : 3, # epoch
#           "early_stopping_rounds" : 50, 
#           "categorical_feature": categorical_feature,
#           "eval_metric" : "F1", # 학습 기준 지표
#           "is_stratified" : True, # stratified 여부
#         }

# params = {"model_name" : "XGBClassifier",
#           "n_splits" : 3, # cross vali 몇개로 나눌것인가
#           "n_estimator" : 3, # epoch
#           "early_stopping_rounds" : 50, 
#           "categorical_feature": categorical_feature,
#           "eval_metric" : "F1", # 학습 기준 지표
#           "is_stratified" : True, # stratified 여부
#         }

params = {"model_name" : "CatBoostClassifier",
          "n_splits": 3, # cross vali 몇개로 나눌것인가
          "n_estimator": 1000, # epoch
          "early_stopping_rounds" : 1000, 
          "categorical_feature": categorical_feature,
          "eval_metric":"F1", # 학습 기준 지표
          "is_stratified": True, # stratified 여부
          "test_size":0.3,
        }

# params = {"model_name" : "AdaBoostClassifier",
#           "n_splits": 3, # cross vali 몇개로 나눌것인가
#           "n_estimator": 3000, # epoch
#           "eval_metric":"F1", # 학습 기준 지표
#           "is_stratified": True, # stratified 여부
#         }

model, test_y = split_train(train_x, train_y, test_x, **params)

In [ ]:
def cv_train(train_x, train_y, test_x, model_name, n_splits=4, n_estimator=200, is_stratified=True, eval_metric="F1",early_stopping_rounds=50, 
             categorical_feature=[], random_state=42, verbose=1, test_size=0.3, save_path=None):
    '''
    - lgbm, xgb, catboost 모델을 cross validation으로 학습한다.
    - 학습한 모델을 가지고 best threshold를 찾는다.
    '''
    from sklearn.metrics import f1_score
    from sklearn.base import clone

    # [data, func setting]
    def custom_f1_LGBMClassifier(y_true, y_pred):
        y_pred = np.where(y_pred >= 0.5 , 1, 0)
        score = f1_score(y_true,y_pred)
        return ("f1_score", score, True)

    def custom_f1_XGBClassifier(y_pred, dtrain):
        y_true = np.array(list(map(int,dtrain.get_label())))
        y_pred = np.where(y_pred >= 0.5 , 1, 0)
        score = f1_score(y_true,y_pred)
        return ("f1_score_xgboost", -score)

    eval_metric_dict_LGBMClassifier = {"F1":custom_f1_LGBMClassifier}
    eval_metric_dict_XGBClassifier = {"F1":custom_f1_XGBClassifier}
    eval_metric_dict_CatBoostClassifier = {"F1":"F1"}

    # [stratified(class 비율맞추기) 유무]
    if is_stratified:
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        data_args = (train_x,train_y)
    else:
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        data_args = (train_x,)

    # [cv]
    models = []
    for i,(tri, vai) in enumerate(cv.split(*data_args)):
        print("="*30,i,"="*30)
        # [모델 선택]
        if model_name=="LGBMClassifier":
            model = LGBMClassifier(n_estimators=n_estimator, 
                                random_state=random_state)
            model.fit(train_x.iloc[tri],train_y.iloc[tri],
                      eval_set=[(train_x.iloc[vai],train_y.iloc[vai])],
                      early_stopping_rounds=early_stopping_rounds,
                      eval_metric=eval_metric_dict_LGBMClassifier[eval_metric],
                      verbose=verbose,
                      categorical_feature=categorical_feature
                      )
        elif model_name=="XGBClassifier":
            model = XGBClassifier(n_estimators=n_estimator, 
                                  random_state=random_state)
            model.fit(train_x.iloc[tri],train_y.iloc[tri],
                      eval_set=[(train_x.iloc[vai],train_y.iloc[vai])],
                      early_stopping_rounds=early_stopping_rounds,
                      eval_metric=eval_metric_dict_XGBClassifier[eval_metric],
                      verbose=verbose)
        elif model_name=="CatBoostClassifier":
            model = CatBoostClassifier(n_estimators=n_estimator, 
                                        random_state=random_state,
                                        eval_metric=eval_metric_dict_CatBoostClassifier[eval_metric],
                                        cat_features=categorical_feature, # 이름이 다른 모델과 다르게 cat_feature인것 주의
                                        one_hot_max_size=4)
            model.fit(train_x.iloc[tri],train_y.iloc[tri],
                      eval_set=[(train_x.iloc[vai],train_y.iloc[vai])],
                      early_stopping_rounds=early_stopping_rounds,
                      verbose=verbose)
        
        models.append(model)
        if save_path:
            with open(save_path+'/model'+str(i)+'.pickle','wb') as fw:
                pickle.dump(model, fw)

    # [find best cv]
    scores = 0
    best_threshold = 0
    cnt_score = 0
    test_y_prob = np.array([0 for i in range(len(test_x))])
    for i,(tri, vai) in enumerate(cv.split(*data_args)):
        y_prob = np.array(models[i].predict_proba(train_x.iloc[vai])[:, 1])
        y_true = np.array(train_y.iloc[vai].iloc[:,0])
        max_threshold, max_score = find_best_threshold(y_true, y_prob, eval_metric) # 0으로 나누어지는 경우가 있다. 이때는 (0.5,-1) 반환

        best_threshold += max_threshold
        if max_score!=-1:
            scores += max_score
            cnt_score += 1
        
        test_y_prob = test_y_prob + models[i].predict_proba(test_x)[:, 1]

    cv_score = scores/cnt_score
    best_threshold = best_threshold/n_splits
    test_y_prob = test_y_prob/n_splits
    print("best score",cv_score,"best_threshold",best_threshold)
    test_y = np.where(test_y_prob >= best_threshold , 1, 0)

    return models, test_y

# params = {"model_name" : "LGBMClassifier",
#           "n_splits" : 3, # cross vali 몇개로 나눌것인가
#           "n_estimator" : 6000, # epoch
#           "early_stopping_rounds" : 600, 
#           "categorical_feature": categorical_feature,
#           "eval_metric" : "F1", # 학습 기준 지표
#           "is_stratified" : True, # stratified 여부
#         }

# params = {"model_name" : "XGBClassifier",
#           "n_splits" : 3, # cross vali 몇개로 나눌것인가
#           "n_estimator" : 3000, # epoch
#           "early_stopping_rounds" : 50, 
#           "categorical_feature": categorical_feature,
#           "eval_metric" : "F1", # 학습 기준 지표
#           "is_stratified" : True, # stratified 여부
#         }

params = {"model_name" : "CatBoostClassifier",
          "n_splits": 3, # cross vali 몇개로 나눌것인가
          "n_estimator": 700, # epoch
          "early_stopping_rounds" : 500, 
          "categorical_feature": categorical_feature,
          "eval_metric":"F1", # 학습 기준 지표
          "is_stratified": True, # stratified 여부
          "test_size":0.3,
          "save_path":path,
        }
        
# params = {"model_name" : "AdaBoostClassifier",
#           "n_splits": 3, # cross vali 몇개로 나눌것인가
#           "n_estimator": 3, # epoch
#           "early_stopping_rounds" : 50, 
#           "categorical_feature": categorical_feature,
#           "eval_metric":"F1", # 학습 기준 지표
#           "is_stratified": True, # stratified 여부
#         }

models, test_y = cv_train(train_x, train_y, test_x, **params)

============================== 1 ==============================
Learning rate set to 0.155546
0:	learn: 0.6016278	test: 0.6026084	best: 0.6026084 (0)	total: 542ms	remaining: 6m 18s
1:	learn: 0.6034067	test: 0.5996306	best: 0.6026084 (0)	total: 842ms	remaining: 4m 53s
2:	learn: 0.6132533	test: 0.6148615	best: 0.6148615 (2)	total: 1.2s	remaining: 4m 37s
3:	learn: 0.6173650	test: 0.6171568	best: 0.6171568 (3)	total: 1.39s	remaining: 4m 2s
4:	learn: 0.6201241	test: 0.6205841	best: 0.6205841 (4)	total: 1.86s	remaining: 4m 19s
5:	learn: 0.6216446	test: 0.6225781	best: 0.6225781 (5)	total: 2.25s	remaining: 4m 20s
6:	learn: 0.6226086	test: 0.6231212	best: 0.6231212 (6)	total: 2.58s	remaining: 4m 15s
7:	learn: 0.6247973	test: 0.6275239	best: 0.6275239 (7)	total: 3.35s	remaining: 4m 50s
8:	learn: 0.6265984	test: 0.6289299	best: 0.6289299 (8)	total: 3.88s	remaining: 4m 58s
9:	learn: 0.6317603	test: 0.6403146	best: 0.6403146 (9)	total: 5.21s	remaining: 5m 59s
10:	learn: 0.6335625	test: 0.6472512	b

In [ ]:
def find_best_cv(train_x, train_y, test_x, models,is_stratified,n_splits,eval_metric,random_state=42,):
    # [stratified(class 비율맞추기) 유무]
    if is_stratified:
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        data_args = (train_x,train_y)
    else:
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        data_args = (train_x,)

    # [find best cv]
    scores = 0
    best_threshold = 0
    cnt_score = 0
    test_y_prob = np.array([0 for i in range(len(test_x))])
    for i,(tri, vai) in enumerate(cv.split(*data_args)):
        y_prob = np.array(models[i].predict_proba(train_x.iloc[vai])[:, 1])
        y_true = np.array(train_y.iloc[vai].iloc[:,0])
        max_threshold, max_score = find_best_threshold(y_true, y_prob, eval_metric) # 0으로 나누어지는 경우가 있다. 이때는 (0.5,-1) 반환

        best_threshold += max_threshold
        if max_score!=-1:
            scores += max_score
            cnt_score += 1
        
        test_y_prob = test_y_prob + models[i].predict_proba(test_x)[:, 1]

    cv_score = scores/cnt_score
    best_threshold = best_threshold/n_splits
    test_y_prob = test_y_prob/n_splits
    print("best score",cv_score,"best_threshold",best_threshold)
    return test_y_prob, best_threshold

# model_name = "XGBClassifier"
# models_xgb=[]
# for i in range(3):
#     with open(path+f'/{model_name}_model{str(i)}.pickle', 'rb') as fr:
#         model = pickle.load(fr)
#     models_xgb.append(model)
# params = {
#     'models' : models_xgb,
#     'is_stratified' : True,
#     'n_splits' : 3,
#     "eval_metric":"F1", # 학습 기준 지표
# }
# test_y_prob_xgb, best_threshold_xgb = find_best_cv(train_x, train_y, test_x, **params)


# model_name="LGBMClassifier"
# models_light=[]
# for i in range(3):
#     with open(path+f'/{model_name}_model{str(i)}.pickle', 'rb') as fr:
#         model = pickle.load(fr)
#     models_light.append(model)
# params = {
#     'models' : models_light,
#     'is_stratified' : True,
#     'n_splits' : 3,
#     "eval_metric":"F1", # 학습 기준 지표
# }
# test_y_prob_light, best_threshold_light = find_best_cv(train_x, train_y, test_x, **params)

model_name="CatBoostClassifier"
models_cat=[]
for i in range(3):
    with open(path+f'/{model_name}_model{str(i)}.pickle', 'rb') as fr:
        model = pickle.load(fr)
    models_cat.append(model)
params = {
    'models' : models_cat,
    'is_stratified' : True,
    'n_splits' : 3,
    "eval_metric":"F1", # 학습 기준 지표
}
test_y_prob_cat, best_threshold_cat = find_best_cv(train_x, train_y, test_x, **params)
test_y = np.where(test_y_prob_cat >= best_threshold_cat , 1, 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()


best score 0.7120441172301599 best_threshold 0.40108797306866656


In [ ]:
def model_predict(train_x, train_y, test_x, score_type):
    '''
    모델들의 cross_validation을 확인하고 가장 스코어가 좋은 모델로 predict한 값을 반환한다. 
    score_type : f1
    '''
    score_model_dict = {}
    scores_XGBClassifier = cross_validate(XGBClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_XGBClassifier["test_"+score_type].mean()] = XGBClassifier()
    
    scores_LGBMClassifier = cross_validate(LGBMClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_LGBMClassifier["test_"+score_type].mean()] = LGBMClassifier()

    scores_AdaBoostClassifier = cross_validate(AdaBoostClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_AdaBoostClassifier["test_"+score_type].mean()] = AdaBoostClassifier()

    scores_DecisionTreeClassifier = cross_validate(DecisionTreeClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_DecisionTreeClassifier["test_"+score_type].mean()] = DecisionTreeClassifier()

    scores_RandomForestClassifier = cross_validate(RandomForestClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_RandomForestClassifier["test_"+score_type].mean()] = RandomForestClassifier()

    scores_RidgeClassifier = cross_validate(RidgeClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_RidgeClassifier["test_"+score_type].mean()] = RidgeClassifier()

    scores_LogisticRegression = cross_validate(LogisticRegression(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_LogisticRegression["test_"+score_type].mean()]  = LogisticRegression()

    scores_CatBoostClassifier = cross_validate(CatBoostClassifier(), train_x, train_y, scoring=[score_type])
    score_model_dict[scores_CatBoostClassifier["test_"+score_type].mean()] = CatBoostClassifier()

    # scores_SVC = cross_validate(SVC(), train_x, train_y, scoring=[score_type]) # 시간이 매우 많이 걸림
    # score_model.append([scores_SVC["test_"+score_type].mean(),SVC()])

    my_score = max(score_model_dict)
    final_model = score_model_dict[my_score]
    final_model.fit(train_x,train_y)
    test_y = final_model.predict(test_x)

    return test_y,final_model,my_score
    
test_y,model,my_score = model_predict(train_x,train_y,test_x,'f1')

In [ ]:
def binary_classification_model(train_x, train_y, test_x, categorical_feature):
    '''
    범주형 컬럼이 포함되어있는 x값을 임베딩처리를 하여 dnn으로 학습
    '''
    from sklearn.preprocessing import LabelEncoder

    input_list = []
    embed_list = []
    for i,col in enumerate(train_x):
        # 범주형이면 벡터화
        if col in categorical_feature:
            # input 
            input_layer = keras.Input(shape=(1,),name=col+"_cat")
            input_list.append(input_layer)

            # 벡터화
            unique_size = len(train_x[col].unique())
            vector_size = int(math.ceil(unique_size/10))
            embedding_layer = layers.Embedding(unique_size+1, vector_size, name=str(i))(input_layer)
            embedding_layer = layers.Reshape(target_shape=(vector_size,))(embedding_layer)
            embed_list.append(embedding_layer)
        else:
            input_layer = keras.Input(shape=(1,),name=col)
            input_list.append(input_layer)
            embed_list.append(input_layer)

    # DNN 
    output = layers.Concatenate()(embed_list)    
    output = layers.Dense(100, activation='relu',name="hidden1")(output)
    output = layers.Dropout(0.5)(output)
    output = layers.Dense(50, activation='relu',name="hidden2")(output)
    output = layers.Dropout(0.3)(output)
    output = layers.Dense(1, activation='sigmoid',name="output_layer")(output)

    model = keras.Model(inputs=input_list,
                        outputs=output)

    model.compile(optimizer="rmsprop",
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    train_x_dnn = []
    test_x_dnn = []
    
    for col in train_x:
        if col in categorical_feature:
            # [train]
            le = LabelEncoder().fit(train_x[col])
            col_le = le.transform(train_x[col])
            train_x_dnn.append(col_le)

            # [test]
            # train에 없는 값은 모두 하나의 값으로
            last_num = len(train_x[col].unique())
            train_set = set(le.classes_.tolist())
            def f(x):
                if x not in train_set:
                    return last_num
                else:
                    return x
            test_x[col] = test_x[col].apply(f)

            # new label 추가
            le_classes = le.classes_.tolist()
            bisect.insort_left(le_classes, last_num)
            le.classes_ = le_classes

            col_le_test = le.transform(test_x[col])
            test_x_dnn.append(col_le_test)
        else:
            train_x_dnn.append(np.array(train_x[col]))
            test_x_dnn.append(np.array(test_x[col]))


    # starts training
    hist=model.fit(train_x_dnn,train_y,
                   batch_size=100,
                   validation_split=0.3,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50)],
                   epochs=1000,
                   verbose=1,
                   shuffle=True)
    
    test_y = (model.predict(test_x_dnn) > 0.5).astype("int32")
    return model, hist, test_y

model, hist, test_y = binary_classification_model(train_x, train_y, test_x, categorical_feature)
grapviz(hist)

In [ ]:
sample_submission["target"] = test_y
my_sub = sample_submission